# Prenalysis to final df

From single_session_analysis.ipynb results of every session. 

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats,signal
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
from IPython.display import HTML, display, Image
import tabulate
import math as m
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.formula.api as smf

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 3
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 4
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.width'] = 2

fday = [1,2,3,4,5,6,7,8,9,10]
fsession = [1,2]

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/gabor/'
path_data = main_path+'/data/jatos_gabor_data/tanda1/'

In [ ]:
rep_list = []
for Day in fday:
    for Ses in fsession:
        path = path_data+'day'+str(Day)+'/session'+str(Ses)+'/'
        data_files = [f for f in os.listdir(path) if f.endswith('_day'+str(Day)+'_session'+str(Ses))]
        # sort files
        subj_data = [int(re.search('%s(.*)%s' % ('', '_day'), f).group(1)) for f in data_files]
        sorted_subj_data = sorted(subj_data)
        index_subj_data = [subj_data.index(elem) for elem in sorted_subj_data]
        sorted_data_files = [data_files[i] for i in index_subj_data]
        # reports, staircase, deterministic and stochastic df for each participant
        Rdf = {}
        for name in sorted_subj_data: 
            Rdf[name] = {}
        ind = -1
        for ses in sorted_data_files:
            ind += 1
            #print(sorted_subj_data[ind])
            data = []
            for line in open(path+ses, 'r'):
                if line.strip():
                    data.append(json.loads(line))
            if Day==1 and Ses==1:
                reports = data[1]
            else:
                reports = data[0]
            part = reports['userID']
            if int(part)>1015 and int(part)<1201:
                part = str(int(part)-186)
            elif int(part)>1201:
                part = str(int(part)-217)

            if reports['sessionID']%2==0:
                session = 2
            else:
                session = 1
                
            Rdf[sorted_subj_data[ind]] = reports   
            del Rdf[sorted_subj_data[ind]]['date']
            rep_list.append(reports)

In [ ]:
Rdf = pd.DataFrame(rep_list)

In [ ]:
Rdf['userID'] = Rdf['userID'].astype(int)

In [ ]:
# rename userID refered to the first deployment userID
secondIDs = [elem for elem in Rdf['userID'].unique() if elem>1015 and elem<1201]
thirdIDs = [elem for elem in Rdf['userID'].unique() if elem>1201]

for ids in secondIDs:
    Rdf.loc[Rdf.userID == ids, 'userID'] = ids-186
for ids in thirdIDs:
    Rdf.loc[Rdf.userID == ids, 'userID'] = ids-217

In [ ]:
stress_list = list(Rdf['stress'])
real_stress = [1-elem for elem in stress_list]
Rdf['real_stress'] = real_stress

In [ ]:
Rdf['sessionID'] = Rdf['sessionID'].astype(int)
Rdf['userID'] = Rdf['userID'].astype(int)

In [ ]:
rep_userID = list(Rdf['userID'])
rep_sessionID = list(Rdf['sessionID'])
rep_user_sessionID = []
for i in range(len(rep_userID)):
    rep_user_sessionID.append(str(rep_userID[i])+'_'+str(rep_sessionID[i]))
Rdf['user_sessionID']=rep_user_sessionID

In [ ]:
Rdf_sin_nan = Rdf

In [ ]:
Rdf['sessionID'] = Rdf['sessionID'].astype(int)
Rdf['userID'] = Rdf['userID'].astype(int)

In [ ]:
path_df = path_results+'across_sessions/'

data_files = [f for f in os.listdir(path_df) if f.startswith('average')]
df = {}
for f in data_files:
    data = [json.loads(line) for line in open(path_df+f, 'r')]
    df[data[0]['sessionID'][0]]=pd.DataFrame.from_dict(data[0])

DF = pd.concat([df[elem] for elem in range(1,len(data_files)+1)])

In [ ]:
DF['sessionID'] = DF['sessionID'].astype(int)
DF['participantID'] = DF['participantID'].astype(int)

In [ ]:
gabor_user_sessionID=[]
gabor_userID = list(DF['participantID'])
gabor_sessionID = list(DF['sessionID'])
for i in range(len(gabor_userID)):
    gabor_user_sessionID.append(str(gabor_userID[i])+'_'+str(gabor_sessionID[i]))
DF['user_sessionID']=gabor_user_sessionID

In [ ]:
# rename userID refered to the first deployment userID
secondIDs = [elem for elem in DF['participantID'].unique() if elem>1015 and elem<1201]
thirdIDs = [elem for elem in DF['participantID'].unique() if elem>1201]
for ids in secondIDs:
    DF.loc[DF.userID == ids, 'participantID'] = ids-186
for ids in thirdIDs:
    DF.loc[DF.userID == ids, 'participantID'] = ids-217

In [ ]:
adf = pd.merge(Rdf,DF,on='user_sessionID',how='outer') 

In [ ]:
df_Hopt = pd.DataFrame(columns=['user_sessionID','HRopt_','SO_HRopt'])
for Day in fday:
    for Ses in fsession:
        sessionid = 2*Day-2+Ses
        path = path_results+'day'+str(Day)+'/session'+str(Ses)+'/'
        optimalH_files = [f for f in os.listdir(path) if f.startswith('optimalH')]
        subj_optimalH = [int(re.search('%s(.*)%s' % ('optimalH_Sub', '_Day'), f).group(1)) for f in optimalH_files]
        sorted_subj_optimalH = sorted(subj_optimalH)
        index_subj_optimalH = [subj_optimalH.index(elem) for elem in sorted_subj_optimalH]
        sorted_optimalH_files = [optimalH_files[i] for i in index_subj_optimalH]
        for filename in sorted_optimalH_files:
            # optimalH
            f = filename
            filename=path+f
            with open(filename) as f:
                data = json.load(f)
            for k, v in data.items():
                globals()[k]=v
            df_Hopt = df_Hopt.append({'HRopt_': HRopt_,'SO_HRopt_': SO_HRopt_,"user_sessionID":part_sessid},ignore_index=True)

In [ ]:
adf = pd.merge(adf,df_Hopt,on='user_sessionID',how='outer') 

In [ ]:
df_PCfit = pd.DataFrame()
for Day in fday:
    for Ses in fsession:
        sessionid = 2*Day-2+Ses
        path = path_results++'day'+str(Day)+'/session'+str(Ses)+'/'
        # Deterministic PC fit
        DOfit_files = [f for f in os.listdir(path) if f.startswith('DO_fit')]
        aux = [f.replace('DO_fit_oo_Sub','') for f in DOfit_files]
        subj_DOfit = [int(f.replace('_Day'+str(Day)+'_Sess'+str(Ses)+'.json','')) for f in aux]
        sorted_subj_DOfit = sorted(subj_DOfit)
        index_subj_DOfit = [subj_DOfit.index(elem) for elem in sorted_subj_DOfit]
        sorted_DOfit_files = [DOfit_files[i] for i in index_subj_DOfit]
        
        # NO PC fit
        NOfit_files = [f for f in os.listdir(path) if f.startswith('NO_fit')]
        auxNO = [f.replace('NO_fit_Sub','') for f in NOfit_files]
        subj_NOfit = [int(f.replace('_Day'+str(Day)+'_Sess'+str(Ses)+'.json','')) for f in auxNO]
        sorted_subj_NOfit = sorted(subj_NOfit)
        index_subj_NOfit = [subj_NOfit.index(elem) for elem in sorted_subj_NOfit]
        sorted_NOfit_files = [NOfit_files[i] for i in index_subj_NOfit]
        
        # Stochastic PC fit
        SOfit_files = [f for f in os.listdir(path) if f.startswith('SO_fit')]
        auxSO = [f.replace('SO_fit_oo_Sub','') for f in SOfit_files]
        subj_SOfit = [int(f.replace('_Day'+str(Day)+'_Sess'+str(Ses)+'.json','')) for f in auxSO]
        sorted_subj_SOfit = sorted(subj_SOfit)
        index_subj_SOfit = [subj_SOfit.index(elem) for elem in sorted_subj_SOfit]
        sorted_SOfit_files = [SOfit_files[i] for i in index_subj_SOfit]
        
        ind = -1
        for filename in sorted_DOfit_files:
            ind += 1
            partid = sorted_subj_DOfit[ind]
            part_sessid = str(partid)+'_'+str(sessionid)
            # deterministic PC fit
            f = filename
            filename=path+f
            with open(filename) as f:
                data = json.load(f)
            for k, v in data.items():
                globals()[k]=v
                
            # psychometric curve NON-optout
            fNO = sorted_NOfit_files[ind]
            filename=path+fNO
            with open(filename) as fNO:
                dataNO = json.load(fNO)
            for k, v in dataNO.items():
                globals()[k]=v     
                
            # stochastic PC fit
            fSO = sorted_SOfit_files[ind]
            filename=path+fSO
            with open(filename) as fSO:
                dataSO = json.load(fSO)
            for k, v in dataSO.items():
                globals()[k]=v   
                
            df_PCfit = df_PCfit.append({'DHR_oo': DHR_oo,'DHL_oo':DHL_oo,'Dsigma_oo':Dsigma_oo,\
                                        'SHR_oo': SHR_oo,'SHL_oo':SHL_oo,'Ssigma_oo':Ssigma_oo,\
                                        'no_sigma':Sigma, 'no_bias':Hno,\
                                        "user_sessionID":part_sessid},ignore_index=True)

In [ ]:
df_PCfit['HmiddleDO']=(df_PCfit['DHR_oo']+df_PCfit['DHL_oo'])/2
df_PCfit['HmiddleSO']=(df_PCfit['SHR_oo']+df_PCfit['SHL_oo'])/2

In [ ]:
adf = pd.merge(adf,df_PCfit,on='user_sessionID',how='outer') 

In [ ]:
adf['risk_av'] = (adf['Dsubj_perf_oo']-adf['Ssubj_perf_oo'])/100.0

In [ ]:
adf['DoverConf'] = (np.abs(2*adf['HRopt_'])-np.abs(adf['DHL_oo']-adf['DHR_oo']))/(np.abs(2*adf['HRopt_'])+np.abs(adf['DHL_oo']-adf['DHR_oo']))

In [ ]:
adf['SoverConf'] = (np.abs(2*adf['SO_HRopt_'])-np.abs(adf['SHL_oo']-adf['SHR_oo']))/(np.abs(2*adf['SO_HRopt_'])+np.abs(adf['SHL_oo']-adf['SHR_oo']))

In [ ]:
# miss a session

excluded_miss = ['1011_11', '1011_18', '1014_12']
adf = adf[~adf['user_sessionID'].isin(excluded_miss)]

In [ ]:
adf_column_names = list(adf.columns)

column_names = [x for x in adf_column_names if x not in ['sessionID_x','sessionID_y','userID','userID_y',\
                                                         'participantID','user_sessionID']]

nan_dict = {}
for i in column_names:
    nan_dict[i] = np.nan

replace = [{'sessionID_x':int(elem.split('_')[1]),'sessionID_y':int(elem.split('_')[1]), \
             'userID':int(elem.split('_')[0]), 'userID_y':int(elem.split('_')[0]),\
            'participantID':int(elem.split('_')[0]),'user_sessionID':elem} for elem in excluded_miss]

for i in range(len(replace)):
    replace[i].update(nan_dict)
    
for elem in replace:
    adf = adf.append(elem, ignore_index=True)

In [ ]:
# exclude 2 sessions because the meedian RT for some difficulties was greater than 2 s

excluded_time = ['1008_2','1009_9']

In [ ]:
# adf = adf[~adf['user_sessionID'].isin(excluded)]

In [ ]:
adf_column_names = list(adf.columns)

column_names = [x for x in adf_column_names if x not in ['userID','sessionID_x','sessionID_y','userID_x','userID_y',\
                                                         'participantID','user_sessionID']]

for ex in excluded_time:
    for col in column_names:
        adf.loc[adf.user_sessionID == ex, col] = np.nan

In [ ]:
adf.groupby(['userID']).size()

In [ ]:
adf[adf['userID']==1011]

In [ ]:
adf.head()

In [ ]:
adf = adf.sort_values(by = ['userID', 'sessionID_y'])

In [ ]:
adf['real_OK_NO_RT'] = adf['OKubj_RT_no']*adf['medianRT']
adf['real_OK_DO_RT'] = adf['OKDsubj_RT_oo']*adf['medianRT']
adf['real_OK_SO_RT'] = adf['OKSsubj_RT_oo']*adf['medianRT']
adf['real_NOK_NO_RT'] = adf['NOKubj_RT_no']*adf['medianRT']
adf['real_DO_RT'] = adf['Dsubj_RT_oo']*adf['medianRT']
adf['real_SO_RT'] = adf['Ssubj_RT_oo']*adf['medianRT']

In [ ]:
adf['sessionID_x'] = adf['sessionID_x'].astype(int)
adf['userID'] = adf['userID'].astype('Int64')

In [ ]:
adf[adf['userID']==1011]

In [ ]:
adf.to_csv(path_results+'preanalyzed.csv',index=False)

In [ ]:
len(adf)